In [1]:
# 📦 Vereiste imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# 🧾 Stap 1 – Laad de data
data = pd.read_excel("..\TrainData.xlsx")

# 🔍 Eerste inspectie
print("Vorm van de data:", data.shape)
print("Kolommen:", data.columns.tolist())
print("Aantal duplicaten:", data.duplicated().sum())
print("Missende waarden per kolom:")
print(data.isnull().sum())

# 🧼 Stap 2 – Dubbele rijen verwijderen
data = data.drop_duplicates()

# 🎯 Stap 3 – Split in features en labels
X = data.drop(columns="label")
y = data["label"]

# 🔁 Zet y om naar numeriek met behoud van pandas Series
label_encoder = LabelEncoder()
y = pd.Series(label_encoder.fit_transform(y), index=y.index)

# ❓ Hoeveel missende waarden blijven over?
print("Totaal aantal missende waarden:", X.isnull().sum().sum())

# ⚠️ Stap 4 – NaNs imputer (mediaan)
X = X.fillna(X.median(numeric_only=True))


#Data normalisatie

X_Scaled = StandardScaler().fit_transform(X)




<>:14: SyntaxWarning: invalid escape sequence '\T'
<>:14: SyntaxWarning: invalid escape sequence '\T'
C:\Users\jelle\AppData\Local\Temp\ipykernel_11536\811751989.py:14: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_excel("..\TrainData.xlsx")


Vorm van de data: (139, 494)
Kolommen: ['PREDICT_original_sf_compactness_avg_2.5D', 'PREDICT_original_sf_compactness_std_2.5D', 'PREDICT_original_sf_rad_dist_avg_2.5D', 'PREDICT_original_sf_rad_dist_std_2.5D', 'PREDICT_original_sf_roughness_avg_2.5D', 'PREDICT_original_sf_roughness_std_2.5D', 'PREDICT_original_sf_convexity_avg_2.5D', 'PREDICT_original_sf_convexity_std_2.5D', 'PREDICT_original_sf_cvar_avg_2.5D', 'PREDICT_original_sf_cvar_std_2.5D', 'PREDICT_original_sf_prax_avg_2.5D', 'PREDICT_original_sf_prax_std_2.5D', 'PREDICT_original_sf_evar_avg_2.5D', 'PREDICT_original_sf_evar_std_2.5D', 'PREDICT_original_sf_solidity_avg_2.5D', 'PREDICT_original_sf_solidity_std_2.5D', 'PREDICT_original_sf_area_avg_2.5D', 'PREDICT_original_sf_area_max_2.5D', 'PREDICT_original_sf_area_min_2.5D', 'PREDICT_original_sf_area_std_2.5D', 'PREDICT_original_sf_volume_2.5D', 'PREDICT_original_of_theta_x', 'PREDICT_original_of_theta_y', 'PREDICT_original_of_theta_z', 'PREDICT_original_hf_min', 'PREDICT_origin

In [41]:
# Feature selection k-best
from sklearn.feature_selection import SelectKBest, f_classif

#k-best
clf = SelectKBest(k=100)
clf.fit(X_Scaled, y)
X_k_best = clf.transform(X_Scaled)
selected_features = clf.get_support(indices=True)

from sklearn.feature_selection import RFECV
from sklearn.neighbors import KNeighborsClassifier



c:\Users\jelle\OneDrive - Delft University of Technology\10007 Machine learning\.conda\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [ 76  77  83  84  85  90  96  97 103 109 110 467 474 475 480 487 488] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\jelle\OneDrive - Delft University of Technology\10007 Machine learning\.conda\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [66]:
class KNNWithFeatureImportance(KNeighborsClassifier):
    def __init__(self, n_neighbors=5, **kwargs):
        super().__init__(n_neighbors=n_neighbors, **kwargs)
        self.feature_importances_ = None
    
    def fit(self, X, y):
        # Call the original fit method
        super().fit(X, y)
        
        # Create a simple feature importance based on feature variance
        # This is just one possible approach - you could use other methods
        rf_model = RandomForestClassifier(n_estimators=50)
        rf_model.fit(X, y)
        
        self.feature_importances_ = rf_model.feature_importances_
        
        return self



In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.neighbors import KNeighborsClassifier

#RFECV met KNN
clf = KNNWithFeatureImportance(n_neighbors=5)
selector = RFECV(clf, step=1, cv=StratifiedKFold(5), scoring='accuracy')
X_selected = selector.fit_transform(X, y)

In [32]:
from sklearn.feature_selection import SequentialFeatureSelector

clf = KNeighborsClassifier(n_neighbors=5)
selector = SequentialFeatureSelector(clf, direction='forward', cv=StratifiedKFold(5))
X_selected = selector.fit_transform(X_k_best, y)



In [33]:
# small test to check if it is better
from sklearn.model_selection import cross_val_score

KNN = KNeighborsClassifier(n_neighbors=5)
scores = cross_validate(KNN, X_selected, y, cv=StratifiedKFold(10), scoring=["roc_auc", "accuracy"])
print(f"AUC = {scores['test_roc_auc'].mean()} and accuracy = {scores['test_accuracy'].mean()}")

AUC = 0.8683673469387756 and accuracy = 0.8269230769230769


In [40]:
# using PCA to reduce the dimensionality of the data
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)  # Reduce to 10 components
X_pca = pca.fit_transform(X_selected)

scores = cross_validate(KNN, X_pca, y, cv=StratifiedKFold(10), scoring=["roc_auc", "accuracy"])
print(f"AUC = {scores['test_roc_auc'].mean()} and accuracy = {scores['test_accuracy'].mean()}")

AUC = 0.8661564625850341 and accuracy = 0.826923076923077


In [35]:
# baseline test
scores = cross_validate(KNN, X, y, cv=StratifiedKFold(10), scoring=["roc_auc", "accuracy"])
print(f"AUC = {scores['test_roc_auc'].mean()} and accuracy = {scores['test_accuracy'].mean()}")

AUC = 0.5285714285714287 and accuracy = 0.5263736263736264


In [ ]:
plt.scatter(X_new[:, 0], X_new[:, 1], c=y, cmap='viridis', edgecolor='k')

In [36]:


X_pca.shape

(139, 30)

In [37]:
X_selected.shape

(139, 50)